# Linear Regression

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from random import uniform,randint, random
from __future__ import division

In [33]:
#Generates a random line in the square [-1,1], and returns the gradient and intercept
def f ():
    #Generate two random points in the square
    (x1,y1) = (uniform(-1,1),uniform(-1,1))
    (x2,y2) = (uniform(-1,1),uniform(-1,1))
    
    m = (y2 - y1) / (x2 - x1)
    c = y1 - m * x1
    
    return (m, c)

#Generate an array of n lists, containing a point in a square, uniformly distributed in [-1,1]
def arrays(n):
    if n == 0:
        return np.array([])
    ar = np.ones((n,3))
    n -= 1
    while n >= 0:
        #x_0 = 1
        ar[n] = [1, uniform(-1,1), uniform (-1,1)]
        n-= 1
    return np.matrix(ar)

#for a given x and f, finds the values of y
def yarray(x, (m, c), n):
    ys = np.zeros((n, 1))
    for i in range(len(x)):
        if x[i, 2] > m * x [i, 1] + c:
            ys[i] = 1
        else:
            ys[i] = -1
    return np.matrix(ys)

#Apply linear regression to the training data set to find g
def g(x, y):
    #w = (x^Tx)^-1 * x^T * y
    w = (x.T * x).I * x.T * y
    return w

#Return the in sample error E_in
def e_in (w, x, y):
    count = 0
    for i in range(len(x)):
        if np.sign(np.dot(x[i], w)) != y[i]:
            count += 1
    
    return count / len(x)

#Apply PLA algorithm, given an initial weight vector w
def PLA (w, x, y):
    #number of iterations
    n = 0
    while n < 1E6:
        #create a list of misclassified points
        misclass = []
        #h(x) = sign(w^T.x)
        for i in range(len(x)):
            if np.sign(np.dot(x[i], w)) != y[i]:
                misclass.append(i)
        n += 1
        #if misclass is empty, we have converged on g
        if not misclass:
            break
        #randomly choose a misclassified point to use to update w
        r = randint(0, len(misclass) - 1)
        #multiply x_i vector by y_i
        adj = np.array(x[misclass[r]]) * np.array(y[misclass[r]])
        #update w
        w = w + adj.T
    return n

#f(x1,x2) = sign(x1^2 + x2^2 - 0.6)
def nonlinear(x):
    y = np.zeros((len(x), 1))
    y = np.matrix(y)
    for i in range(len(x)):
        y[i] = np.sign(x[i,1]**2 + x[i,2]**2 - 0.6)
        #Add random noise in 10% of data by flipping the sign
        if random() < 0.1:
            y[i] = -1 * y[i]
    return y

#Adds additional values to the x arrays so we can use regression on a 'linear' data set
def transform(n):
    if n == 0:
        return np.array([])
    ar = np.ones((n,6))
    n -= 1
    while n >= 0:
        #x_0 = 1
        x1 = uniform(-1,1)
        x2 = uniform(-1,1)
        ar[n] = [1, x1, x2, x1 * x2, x1**2, x2**2]
        n-= 1
    return np.matrix(ar)

#Calculates y values, slightly more efficient as we already calculated x1^2 and x2^2
def transformedlinear(x):
    y = np.zeros((len(x), 1))
    y = np.matrix(y)
    for i in range(len(x)):
        y[i] = np.sign(x[i,4] + x[i,5] - 0.6)
        #Add random noise in 10% of data by flipping the sign
        if random() < 0.1:
            y[i] = -1 * y[i]
    return y

In [76]:
#Find average in sample error over 1000 runs
N = 100
repeat = 1000
count = 0
for i in range(repeat):
    (m, c) = f()
    x = arrays(N)
    y = yarray(x, (m, c), N)
    w = g(x, y)
    E_in = e_in(w, x, y)
    count += E_in
print "{}".format(count / repeat)

0.03804


In [56]:
#Find average out of sample error over 1000 runs
N = 100
repeat = 1000
count = 0
for i in range(repeat):
    (m, c) = f()
    x = arrays(N)
    y = yarray(x, (m, c), N)
    w = g(x, y)
    outofsample = arrays(1000)
    y_out = yarray(outofsample, (m, c), 1000)
    for i in range(len(outofsample)):
        if np.sign(np.dot(outofsample[i], w)) != y_out[i]:
            count += 1
print "{}".format(count / (repeat * 1000))

0.047969


In [92]:
#Use linear regression as starter for classification, and find average number of steps needed to converge using PLA
#afterwards
N = 10
repeat = 1000
count = 0
for i in range(repeat):
    (m, c) = f()
    x = arrays(N)
    y = yarray(x, (m, c), N)
    w = g(x, y)
    pla = PLA(w, x, y)
    count += pla
print "{}".format(count / repeat)

5.404


In [29]:
#Use linear regression to find classification for non linear input
N = 1000
repeat = 1000
count = 0
for i in range(repeat):
    x = arrays(N)
    y = nonlinear(x)
    w = g(x, y)
    E_in = e_in(w, x, y)
    count += E_in
print "{}".format(count / repeat)

0.504714


In [40]:
#Use linear regression to classify non linear input which is initially transformed
N = 1000
repeat = 10
count = np.zeros((6, 1))
for i in range(repeat):
    x = transform(N)
    y = transformedlinear(x)
    w = g(x, y)
    count = count + w
print "{}".format(count / repeat)

[[-0.99895963]
 [ 0.00336274]
 [-0.00655032]
 [-0.04116103]
 [ 1.55258916]
 [ 1.58883062]]


In [44]:
#Use same method of transformation as before, but now estimate out of sample error
N = 1000
repeat = 1000
count = 0
for i in range(repeat):
    x = transform(N)
    y = transformedlinear(x)
    w = g(x, y)
    outofsample = transform(1000)
    y_out = transformedlinear(outofsample)
    for i in range(len(outofsample)):
        if np.sign(np.dot(outofsample[i], w)) != y_out[i]:
            count += 1
print "{}".format(count / (1000 * repeat))

0.126328
